In [74]:
import pandas as pd
import numpy as np
import os
import json
# import gui
from collections import Counter
import re
import math as m

pd.options.display.max_columns = 20000

# Create a pandas dataframe of all cards
![Alt Text](https://media.giphy.com/media/vFKqnCdLPNOKc/giphy.gif)

### Create initial dataframe and perform some preprocessing tasks

In [75]:
lexicon = pd.read_feather('mtg_card_database')
colors = ["W", "U", "B", "R", "G", "C", "P",]

In [76]:
# Only keep the most recent arena set, dropping all reprints
lexicon = lexicon.sort_values(by='arena_id', ascending=False).copy()
lexicon = lexicon.drop_duplicates(subset=["name"])

# Set the index for card names to easily search
def set_index(lexicon):
    lexicon = lexicon.set_index("name", drop=False)
    lexicon.index.name = "card_name"
    return lexicon
lexicon = set_index(lexicon)

# Drop unused columns
lexicon = lexicon.drop(columns=['id', 'power', 'toughness', 'color_identity', 'set', 'keywords', 'flavor_text', 'edhrec_rank', 'arena_id', 'loyalty'], inplace=False)

In [77]:
# Clean mana_cost
def clean_mana_cost(s):
    """Cleans the mana_cost column by removing curly braces {} and by converting any numbers in mana cost to 1s representation

    Args:
        s (string): the mana_cost string 

    Returns:
        string: cleaned mana cost
    """
    # If s is not nonetype
    try: 
        s = s.replace("{", "").replace("}", "")

        # If the first character of s is a number
        try:
            num = int(s[0])
            s = s.replace(s[0], '1' * num)
        except:
            pass
    except:
        pass

    return s

lexicon['mana_cost'] = lexicon['mana_cost'].apply(lambda s: clean_mana_cost(s))

## Fix Dual-Faced Cards

### Extract card data for both sides

In [78]:
# Let's first create a dataframe that just has the card name and the column 'card_faces'
double_cards_df = lexicon[['name','card_faces']].dropna()

# DFC are a list of two dictionaries [ {k:v, k2:v2}, {k3:v3} ]
def split_card_faces(row, face_number):
    face = row[face_number]
    return face

face_one = 0
face_two = 1
double_cards_df['face1'] = double_cards_df['card_faces'].apply(lambda row: split_card_faces(row, face_one))
double_cards_df['face2'] = double_cards_df['card_faces'].apply(lambda row: split_card_faces(row, face_two))

# # Now let's drop the column 'card_faces'
# double_cards_df = double_cards_df.copy()
# double_cards_df.drop("card_faces",axis=1)

# We now go into each key within the dictionary of face1 and face2 and separate them into columns
try:
    double_cards_df[double_cards_df['face1'].apply(pd.Series).columns + "_1"] = double_cards_df['face1'].apply(pd.Series)
    double_cards_df[double_cards_df['face2'].apply(pd.Series).columns + "_2"] = double_cards_df['face2'].apply(pd.Series)
except:
    pass

# Define a list of columns we want to keep from the DFCs
cols_to_keep = ['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
                'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']

# For each column in the dataframe, if it's not a selected column, we drop it
for col in double_cards_df.columns:
    if col not in cols_to_keep:
        double_cards_df.drop(col, axis=1, inplace=True)

# We now need to consolidate the 2 oracle texts into 1, we join them together
double_cards_df['oracle_text_dobles'] = double_cards_df['oracle_text_1'] + "\n" + double_cards_df['oracle_text_2']

# Clean the mana costs
double_cards_df['mana_cost_1'] = double_cards_df['mana_cost_1'].apply(lambda s: clean_mana_cost(s))
double_cards_df['mana_cost_2'] = double_cards_df['mana_cost_2'].apply(lambda s: clean_mana_cost(s))

In [79]:
# We now merge them by card name
lexicon = lexicon.merge(double_cards_df, on=["name"], how="left")

In [80]:
# If oracle_text is empty (meaning it's a double faced card), we replace it with our 'oracle_text_dobles' column
lexicon['oracle_text'] = np.where(lexicon['oracle_text'].isna(),lexicon['oracle_text_dobles'],lexicon['oracle_text'])


In [81]:
# Drop secret lair dual faced cards which should be single faced cards
tmpdf = lexicon[lexicon['name'].str.contains("//")]
mask = tmpdf[tmpdf['type_line'].isna()]
lexicon = lexicon[~lexicon['name'].isin(mask['name'])]
lexicon = lexicon.reset_index()

In [82]:
# We use this script to replace Nulls with "None"
# lexicon[['oracle_text_1','oracle_text_2']] = lexicon[['oracle_text_1','oracle_text_2']]

# try:
#     lexicon[['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
#                 'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']] = lexicon[['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
#                                                                                         'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']]
# except:
#     pass

# Now that we have our oracle text from the 2 card sides joined together, we want to use it to replace
# the actual "oracle_text" from the original dataframe, which is actually empty

# lexicon = set_index(lexicon)

# # And now that column is useless so we drop it
# lexicon = lexicon.drop("oracle_text_dobles",axis=1)

### Average the mana cost of both sides as well as the cmc

In [83]:
def dfc_avg_mana_cost_and_cmc(row):
    """Averages the mana cost for dual-faced cards and the cmc

    Args:
        row (pd.Series): one row of the lexicon df

    Returns:
        tuple of strings: averaged mana cost and average cmc
    """
    # Only for dual faced cards
    if isinstance(row.card_faces, type(np.ndarray(0))):
        # Initialize return value
        mana_cost = ''
        # Only for DFC with mana costs
        try:
            # Count the values of each mana symbol and add them together into a dictionary
            s1 = Counter(row.mana_cost_1)
            s2 = Counter(row.mana_cost_2)
            s3 = s1+s2
            if s3 == s1:
                row.mana_cost = row.mana_cost_1
                return row
            elif s3 == s2:
                row.mana_cost = row.mana_cost_2
                return row
            s4 = dict(s3)
            # Divide all the values by 2 rounded up for Mana symbols and down for generic costs
            for k in s4.keys():
                # If there is a generic casting cost
                try:
                    int(k)
                    s4[k] = max(m.floor(s4[k] / 2), 1)
                # If there is a colored mana symbol
                except:
                    s4[k] = m.ceil(s4[k] / 2)
                # concatenate the dict back into a string
                mana_cost += (k * s4[k])
            # Average cmc which is a return value
            avg_cmc = sum(s3.values()) / 2
            row.cmc = avg_cmc
            row.mana_cost = mana_cost
            # Return the averaged mana cost as a string
            return row
        except:
            pass
    # If its not a DFC then just return the mana_cost already present
    return row

lexicon = lexicon.apply(dfc_avg_mana_cost_and_cmc, axis=1)

### Create columns for casting cost pips
note: Phyrexian mana is represented as a P

In [93]:
def count_pips(mana_cost):
    """Applied to the mana_cost column in lexicon to create columns for the number of pips in that color for each color

    Args:
        mana_cost (string): the mana cost as a str

    Returns:
        Counter: histogram of mana cost pips
    """
    pips = Counter(mana_cost)
    if '/' in pips:
        print(pips)

    return pips

lexicon = lexicon.copy()
for color in colors:
    row_name = f'cast_cost_{color}'
    lexicon.loc[:, f'{row_name}'] = lexicon['mana_cost'].apply(lambda row: count_pips(row)[f'{color}'])

Counter({'U': 2, 'R': 2, '/': 1})
Counter({'G': 2, '/': 2, 'W': 2, '1': 1, 'P': 1})
Counter({'1': 2, 'G': 2, '/': 2, 'U': 2, 'P': 1})
Counter({'U': 3, '/': 2, 'W': 1, 'B': 1})
Counter({'W': 3, '/': 2, 'G': 1, 'U': 1})
Counter({'R': 3, '/': 2, '1': 1, 'B': 1, 'G': 1})
Counter({'G': 3, '/': 2, 'R': 1, 'W': 1})
Counter({'B': 3, '1': 2, '/': 2, 'U': 1, 'R': 1})
Counter({'1': 3, 'U': 1, '/': 1, 'P': 1})
Counter({'1': 3, 'B': 2, '/': 2, 'G': 2})
Counter({'U': 2, '/': 2, 'R': 2})
Counter({'1': 1, 'G': 1, '/': 1, 'U': 1})
Counter({'R': 2, '/': 2, 'W': 2, '1': 1})
Counter({'W': 2, '/': 2, 'B': 2, '1': 1})
Counter({'W': 2, 'B': 2, '/': 1})
Counter({'1': 4, 'W': 2, '/': 2, 'B': 2})
Counter({'1': 3, 'R': 2, 'W': 2, '/': 1})
Counter({'G': 2, '/': 2, 'U': 2, '1': 1})
Counter({'G': 2, 'U': 2, '1': 1, '/': 1})
Counter({'U': 2, '/': 2, 'R': 2})
Counter({'B': 2, '/': 2, 'G': 2, '1': 1})
Counter({'G': 4, '/': 4, 'U': 4})
Counter({'R': 1, '/': 1, 'W': 1})
Counter({'U': 2, 'R': 2, '1': 1, '/': 1})
Counter(

### Create columns for produced mana colors of non-land cards

In [85]:
def count_produced_mana(row):
    """Applied to the produced_mana column, returns the colors produced by a card

    Args:
        row (List[str]): 

    Returns:
        prod_mana: _description_
    """    
    return Counter(row)

lexicon = lexicon.copy()
for color in colors:
    row_name = f'produces_{color}'
    lexicon.loc[:, f'{row_name}'] = lexicon['produced_mana'].apply(lambda row: count_produced_mana(row)[f'{color}'])

## Hybrid Mana Costs
Hybrid mana is represented as U/R
- U/R should have a casting cost of 0.5U and 0.5R

In [100]:
lexicon[lexicon['mana_cost'].str.contains('/')]
# print(tmp[tmp['name'].str.contains('K\'rrik, Son of Yawgmoth')])

,index,name,mana_cost,cmc,type_line,oracle_text,colors,produced_mana,rarity,card_faces,mana_cost_1,name_1,oracle_text_1,type_line_1,mana_cost_2,name_2,oracle_text_2,type_line_2,oracle_text_dobles,cast_cost_W,cast_cost_U,cast_cost_B,cast_cost_R,cast_cost_G,cast_cost_C,cast_cost_P,produces_W,produces_U,produces_B,produces_R,produces_G,produces_C,produces_P
128,128,A-Maelstrom Muse,UU/RR,4.0,Creature — Djinn Wizard,"Flying\nWhenever Maelstrom Muse attacks, the n...","[R, U]",None,uncommon,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,0,2,0,0,0,0,0,0,0,0,0,0
549,549,"Ajani, Sleeper Agent",1GG/W/PW,4.0,Legendary Planeswalker — Ajani,"Compleated ({G/W/P} can be paid with {G}, {W},...","[G, W]",None,mythic,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,2,0,1,0,0,0,0,0,0,0
1165,1165,"Tamiyo, Compleated Sage",11GG/U/PU,5.0,Legendary Planeswalker — Tamiyo,"Compleated ({G/U/P} can be paid with {G}, {U},...","[G, U]",None,mythic,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,0,0,2,0,1,0,0,0,0,0,0,0
1234,1234,"Toluz, Clever Conductor",W/UUU/B,3.0,Legendary Creature — Human Rogue,"When Toluz, Clever Conductor enters the battle...","[B, U, W]",None,rare,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,1,0,0,0,0,0,0,0,0,0,0,0
1241,1241,"Rigo, Streetwise Mentor",G/WWW/U,3.0,Legendary Creature — Cat Citizen,"Rigo, Streetwise Mentor enters the battlefield...","[G, U, W]",None,rare,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26894,26905,Sootwalkers,11B/RB/R,4.0,Creature — Elemental Rogue,Sootwalkers can't be blocked by white creatures.,"[B, R]",None,common,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2,2,0,0,0,0,0,0,0,0,0,0
27033,27044,Cultbrand Cinder,1111B/R,5.0,Creature — Elemental Shaman,"When Cultbrand Cinder enters the battlefield, ...","[B, R]",None,common,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,1,0,0,0,0,0,0,0,0,0,0
27160,27171,Sangrite Backlash,B/GR,2.0,Enchantment — Aura,Enchant creature\nEnchanted creature gets +3/-3.,"[B, G, R]",None,common,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,1,1,0,0,0,0,0,0,0,0,0
27177,27188,Pyrrhic Revival,111W/BW/BW/B,6.0,Sorcery,Each player returns each creature card from th...,"[B, W]",None,rare,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,3,0,0,0,0,0,0,0,0,0,0,0


- Mana_Costs
    - Phyrexian Mana
    - Dual Color (either red or white mana can be used to pay for RW)
    - X cost (average value for X is math.floor(4 / num_x))
- Card Selection
    - define RegEx pattern
    - Create method
- Cards which make treasure
- Beautiful Soup
    - How to use it
    - How big do we want our dataset
    - How do we store the dataframes???